In [14]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
# from sklearn.naive_bayes import GaussianNB
import sys
sys.path.append('../')
from utils.utils import *

In [11]:
X, y = get_training()
X.dropna(inplace=True)
y = y.loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
def get_stacking():
    level0 = list()
    logreg = joblib.load('logistic_regression_model.pkl')
    xgb = joblib.load('XGB_model.pkl')
    # nba = GaussianNB()
    level0.append(('Logisitic Regression', logreg))
    level0.append(('Extreme Gradient Boosting', xgb))
    # level0.append(('Naive Bayes', nba))
    level1 = LogisticRegression(multi_class='multinomial')
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [16]:
stacking_clf = get_stacking()
stacking_clf.fit(X_train, y_train)

/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/srijanchaudhuri/opt/miniconda3/envs/stat441/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Plea

StackingClassifier(cv=5,
                   estimators=[('Logisitic Regression',
                                LogisticRegression(multi_class='multinomial')),
                               ('Extreme Gradient Boosting',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_typ...
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_class=5,
                                              num_parallel_tree=None, ...)),
                               ('Naive Bayes', GaussianNB())],
                   final_estimator=LogisticRegression(multi_class='multinomial'))

In [17]:
from sklearn.metrics import log_loss, accuracy_score

y_pred_proba = stacking_clf.predict_proba(X_test)
y_pred = stacking_clf.predict(X_test)

logloss = log_loss(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)

print(f"Log Loss: {logloss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Log Loss: 0.8784
Accuracy: 0.6337
